In [14]:
# Import necessary libraries
import dspy
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Initialize the OpenAI language model
cheap_lm = dspy.LM("openai/gpt-5-mini", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000, reasoning_effort="low")
smart_lm = dspy.LM("openai/gpt-5", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000, reasoning_effort="high")

# Define the Comedian class signature
class Comedian(dspy.Signature):
    """
    Tell me a funny joke.
    """
    topic: str = dspy.InputField(description="The message to analyze")
    joke: str = dspy.OutputField(description="A funny joke about the topic")

# Define the comedian program
comedian = dspy.Predict(Comedian)

# Create instances of Comedian with different LMs
with dspy.context(lm=cheap_lm):
    cheap_response = comedian(topic="programming")

with dspy.context(lm=smart_lm):
    smart_response = comedian(topic="programming")

print("Cheap LM response:")
print(cheap_response.joke)

print("\nSmart LM response:") 
print(smart_response.joke)

Cheap LM response:
Why do programmers prefer dark mode? Because light attracts bugs — and they already have enough features labeled "undocumented."

Smart LM response:
There are only two hard things in programming: cache invalidation, naming things, and off-by-one errors.
